In [2]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import imputation
import cross_validate as scv
import log_worker as slog
import plotting as splt
from model_analsyis import nested_cross_validation_analysis as ncv_analysis
import evaluation as evaluate
import mathx
from sklearn import metrics

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer as Imputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# ************************** GLOBAL PARAMETERS ******************************************
data_dir = os.path.join(os.getcwd(),'data')
input_data_file = os.path.abspath('temp__mc80_imputed_normalized_2019-08-23.csv')
output_dir = os.path.join(data_dir, 'interim')
figure_dir = output_dir
metrics_output_file = os.path.join(output_dir,'scoring_metrics.txt')
store_prediction_probs = True
prediction_prob_dir = os.path.join(output_dir, 'prediction_probabilities')
selected_features_file = os.path.join(output_dir,'selected_features.txt')
seed = 287462
num_folds = 10  # number of stratified folds for outside loop
shuffle_on_split = False  # shuffle data before splitting into folds
# number of features for feature selection, recommended number positive examples in training/ 10
# if set to -1 use all features
num_features = -1
date_string='2019-08-23'
file_prefix='temp_'

slog.log_items("Global Parameters\n", metrics_output_file,
               num_folds=num_folds,
               num_features=num_features,
               shuffle_on_split=shuffle_on_split,
               seed=seed)

import warnings
warnings.filterwarnings('ignore')

# ************************* GLOBAL PARAMETERS END *************************************

In [7]:
# import data

cases= pd.read_csv(os.path.join(os.path.join(data_dir,'raw'),'CASE_FILE.csv'))

controls = pd.read_csv(os.path.join(os.path.join(data_dir,'raw'),'CONTROL_FILE.csv'))


### ***Preprocessing Section***

In [8]:
# Preprocessing Section
cols_to_drop = []

# column names for features requiring normalization
cols_to_norm = ['gest_age', 'age', 'wbc', 'hgb', 'it_ratio',
       'capPH', 'bicarb', 'glucose', 'creatinine', 'platelet_count', 'hr',
       'rr', 'temp', 'sbp', 'dbp', 'map', 'weight', 'fio2',
       'hr_delta', 'rr_delta', 'mabp_delta',
       'temp_delta']
# Sepsis Groups
# Group 1: Culture Positive sepsis: culture positive, minimum 5 days antibiotic treatment
# Group 2: Negative Sepsis: Culture negative, <72 hours antibiotic treatment
# Group 3: Clinical Sepsis: Culture negative, >120 hours of antibiotic treatment
# sepsis groups to use in positive samples (cases):
CASE_GRPS = [1,3]

# control group data is from non-septic periods for same individuals in case groups
# if using controls file with unspecified groups, set to None
CONTROL_GRPS = None
missing_cutoff = 80

cases['sepsis'] = 1
controls['sepsis'] = 0

# select data from specified sepsis groups
if CONTROL_GRPS is None:
    controls['sepsis_group'] = -1
    CONTROL_GRPS = [-1]
X = pd.concat([cases[cases['sepsis_group'].isin(CASE_GRPS)],
               controls[controls['sepsis_group'].isin(CONTROL_GRPS)]],sort=True)

of = os.path.join(data_dir ,'case_control_stats.txt')
slog.log_line("samples count: {0}\n".format(len(X)), of)
y = X['sepsis']
slog.log_line('target counts\n{0}\n'.format(y.value_counts()), of)
slog.log_line('Incidence Rate (percent): {0:.3f}\n'.format(100 * np.sum(y) / float(len(y))), of)

# Imputation
missing_percentages = imputation.missing_percents(X)
slog.log_dictionary(missing_percentages, 'Missing Data Percentages\n', False,
                          os.path.join(data_dir ,'missing_data_percents.txt'))

# drop columns with missing percentage over threshold
#cols_to_drop = []
for k,v in missing_percentages.items():
    if v > missing_cutoff:
        cols_to_drop.append(k)

for c in cols_to_drop:
    X = X.drop(c,axis=1)
    if c in cols_to_norm:
        cols_to_norm.remove(c)
        
        # impute missing values
imp = Imputer(strategy='mean')
Ximp = imp.fit_transform(X)
cnames = X.columns
X = pd.DataFrame(data=Ximp, columns=cnames)

# Normalization
X[cols_to_norm] = X[cols_to_norm].apply(lambda x: (x-x.mean())/x.std())

## Store processed data
X.to_csv('{0}_mc{1}_imputed_normalized_{2}.csv'.
                      format(file_prefix,missing_cutoff,date_string), index=False)


### ***Trian and validate MODELS***

In [12]:

# import preprocessd data
all_data = pd.read_csv(input_data_file).sample(frac=1, random_state=seed)
y = all_data['sepsis']
X = all_data.drop('sepsis', axis=1).drop('sepsis_group', axis=1)

# Use stratified K-fold to get data split indices
skf = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=shuffle_on_split)
folds = {}
fold_idx = 0
for train_split, test_split in skf.split(X,y):
    folds[fold_idx] = test_split
    fold_idx += 1

# evaluate models
kn = num_features
if num_features == -1:
    kn = len(X.columns)

# wrapper method to enable passing a random state for scoring method of feature selector
def seeded_mutual_info_classif(X, y):
    return mutual_info_classif(X,y, random_state=seed)

feature_selector = SelectKBest(seeded_mutual_info_classif, k=kn)

if store_prediction_probs and not os.path.exists(prediction_prob_dir):
    os.makedirs(prediction_prob_dir)


In [11]:

# ************************* Model evaluations ****************************************
# --------------------------   logistic regression --------------------------------------------
parameter_candidates = [{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]

model = LogisticRegression(penalty='l2',class_weight='balanced', random_state=seed)
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "Logistic Regression", "LR",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "LR_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "LR_targets.csv"), seed=seed,
             selected_features_file=selected_features_file,
             feature_coefs_file=os.path.join(output_dir,"LR_coefs.csv")
             )

Starting run 0 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 1 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 2 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 3 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 4 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 5 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 6 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 7 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 8 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Starting run 9 of 10 for LogisticRegression
	Starting grid search ....
	Grid search complete ....
Logistic Regression


In [6]:

#--------------------------  support vector machine ----------------------------------------------
parameter_candidates = [{'C': [0.01, 0.1, 1, 10, 100],
                         'gamma': [0.01, 0.1, 1, 10, 100]}]
model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=seed)
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "SVM (RBF)", "SVM-RBF",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "SVM_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "SVM_targets.csv"), seed=seed, n_jobs=4)


Starting run 0 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 1 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 2 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 3 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 4 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 5 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 6 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 7 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 8 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
Starting run 9 of 10 for SVC
	Starting grid search ....
	Grid search complete ....
SVM (RBF)
accuracy:	mean=0.796	std=0.030	range=[0.748,0.851]
f1:	mean=0.651	std=0.044	range=[0.602,0.738]
sensitivity:	mean=0.749	std=0.052	range=[0.676,0.842]
specificit

In [7]:

#  --------------------------  Gaussian NB -------------------------------------------------------
model = GaussianNB()
metric_values, fpr_scores, tpr_scores, precision_scores, recall_scores = \
   scv.nested_cross_validate(model, None, folds, X, y, feature_selector=feature_selector)
ncv_analysis(model, None, folds, X, y, feature_selector, "Naive Bayes", "NaiveBayes",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "NaiveBayes_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "NaiveBayes_targets.csv"), seed=seed)

Starting run 0 of 10 for GaussianNB
Starting run 1 of 10 for GaussianNB
Starting run 2 of 10 for GaussianNB
Starting run 3 of 10 for GaussianNB
Starting run 4 of 10 for GaussianNB
Starting run 5 of 10 for GaussianNB
Starting run 6 of 10 for GaussianNB
Starting run 7 of 10 for GaussianNB
Starting run 8 of 10 for GaussianNB
Starting run 9 of 10 for GaussianNB
Starting run 0 of 10 for GaussianNB
Starting run 1 of 10 for GaussianNB
Starting run 2 of 10 for GaussianNB
Starting run 3 of 10 for GaussianNB
Starting run 4 of 10 for GaussianNB
Starting run 5 of 10 for GaussianNB
Starting run 6 of 10 for GaussianNB
Starting run 7 of 10 for GaussianNB
Starting run 8 of 10 for GaussianNB
Starting run 9 of 10 for GaussianNB
Naive Bayes
accuracy:	mean=0.809	std=0.009	range=[0.796,0.824]
f1:	mean=0.527	std=0.048	range=[0.408,0.594]
sensitivity:	mean=0.424	std=0.066	range=[0.270,0.526]
specificity:	mean=0.941	std=0.022	range=[0.900,0.982]
precision:	mean=0.720	std=0.058	range=[0.640,0.833]
ROC-AUC:	mea

In [8]:

#  --------------------------  Gaussian Process -------------------------------------------
model = GaussianProcessClassifier(random_state = seed)
metric_values, fpr_scores, tpr_scores, precision_scores, recall_scores = \
    scv.nested_cross_validate(model, None, folds, X, y, feature_selector=feature_selector)
ncv_analysis(model, None, folds, X, y, feature_selector, "Gaussian Process", "GaussianProcess",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "GaussianProcess_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "GaussianProcess_targets.csv"), seed=seed)


Starting run 0 of 10 for GaussianProcessClassifier
Starting run 1 of 10 for GaussianProcessClassifier
Starting run 2 of 10 for GaussianProcessClassifier
Starting run 3 of 10 for GaussianProcessClassifier
Starting run 4 of 10 for GaussianProcessClassifier
Starting run 5 of 10 for GaussianProcessClassifier
Starting run 6 of 10 for GaussianProcessClassifier
Starting run 7 of 10 for GaussianProcessClassifier
Starting run 8 of 10 for GaussianProcessClassifier
Starting run 9 of 10 for GaussianProcessClassifier
Starting run 0 of 10 for GaussianProcessClassifier
Starting run 1 of 10 for GaussianProcessClassifier
Starting run 2 of 10 for GaussianProcessClassifier
Starting run 3 of 10 for GaussianProcessClassifier
Starting run 4 of 10 for GaussianProcessClassifier
Starting run 5 of 10 for GaussianProcessClassifier
Starting run 6 of 10 for GaussianProcessClassifier
Starting run 7 of 10 for GaussianProcessClassifier
Starting run 8 of 10 for GaussianProcessClassifier
Starting run 9 of 10 for Gaussi

In [9]:

#  --------------------------  Random Forest --------------------------------------------
parameter_candidates = [{'n_estimators': [10, 50, 100, 200],
                         'criterion': ['gini','entropy'],
                         'max_depth': [2, 4, 6]}]
model = RandomForestClassifier(random_state=seed, class_weight='balanced')
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "Random Forest", "RandomForest",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "RandomForest_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "RandomForest_targets.csv"), seed=seed)

Starting run 0 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 1 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 2 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 3 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 4 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 5 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 6 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 7 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 8 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 9 of 10 for RandomForestClassifier
	Starting grid search ....
	Grid s

In [21]:

#  --------------------------  AdaBoost -------------------------------------------------
parameter_candidates = [{'base_estimator': [DecisionTreeClassifier(),
                                            LogisticRegression(class_weight='balanced', random_state=seed),
                                            SVC(kernel='rbf', probability=True,class_weight='balanced', random_state=seed)],
                         'n_estimators': [50, 100],
                         'learning_rate': [1.0, 0.5, 0.1]}]
model = AdaBoostClassifier(random_state=seed)
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "AdaBoost", "AdaBoost",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "AdaBoost_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "AdaBoost_targets.csv"), seed=seed)


Starting run 0 of 10 for AdaBoostClassifier
	Starting grid search ....


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "J:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-3dea53493e9c>", line 11, in <module>
    os.path.join(prediction_prob_dir, "AdaBoost_targets.csv"), seed=seed)
  File "C:\Users\RR\Documents\Python Scripts\sepsis\model_analsyis.py", line 32, in nested_cross_validation_analysis
    feature_coefs_file=feature_coefs_file)
  File "C:\Users\RR\Documents\Python Scripts\sepsis\cross_validate.py", line 97, in nested_cross_validate
    gs.fit(X_cv, y_cv)
  File "J:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 687, in fit
    self._run_search(evaluate_candidates)
  File "J:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 1148, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "J:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 666, in evalua

KeyboardInterrupt: 

In [13]:

#  --------------------------  KNN Classifier ---------------------------------------
parameter_candidates = [{'n_neighbors': [5, 10],
                         'weights': ['uniform', 'distance']}]
model = KNeighborsClassifier()
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "KNN", "KNN",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "KNN_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "KNN_targets.csv"), seed=seed)

Starting run 0 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 1 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 2 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 3 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 4 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 5 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 6 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 7 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 8 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 9 of 10 for KNeighborsClassifier
	Starting grid search ....
	Grid search complete ....


In [14]:

# --------------------------  Gradient Boosting -------------------------------------------
# wrapper class to fix sklearn bug
class init:
    def __init__(self, est):
        self.est = est
    def predict(self, X):
        return self.est.predict_proba(X)[:,1][:,np.newaxis]
    def fit(self, X, y, *kwarg):
        self.est.fit(X, y)


model = GradientBoostingClassifier(random_state=seed)
parameter_candidates = [{'n_estimators': [50, 100, 200],
                         'max_depth': [3, 5, 10],
                         # 'init': [None,
                         #          init(DecisionTreeClassifier(class_weight='balanced')),
                         #          init(LogisticRegression(class_weight='balanced', random_state=seed)),
                         #          init(SVC(kernel='rbf', probability=True,class_weight='balanced'))]
                         }
                        ]
ncv_analysis(model, parameter_candidates, folds, X, y, feature_selector, "Gradient Boost", "GradBoost",
             figure_dir, metrics_output_file, store_prediction_probs,
             os.path.join(prediction_prob_dir, "GradBoost_pred_probs.csv"),
             os.path.join(prediction_prob_dir, "GradBoost_targets.csv"), seed=seed)


Starting run 0 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 1 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 2 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 3 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 4 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 5 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 6 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 7 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 8 of 10 for GradientBoostingClassifier
	Starting grid search ....
	Grid search complete ....
Starting run 9 of 10 for GradientBoostingClass

### ***Postprocessing section***

In [19]:
print(data_dir)

..\data


In [20]:
 # Postprocessing section
   
   
  #  ************************** GLOBAL PARAMETERS ******************************************
data_dir = os.path.join(os.getcwd(),'data')

input_data_dir = os.path.join(data_dir, "results",
                              "PATH/TO/PREDICTION/PROBABILITES/FILE",
                              "prediction_probabilities")


_prob_file = os.path.join(input_data_dir,"{0}_pred_probs.csv")
_targ_file = os.path.join(input_data_dir,"{0}_targets.csv")
file_prefixes = ['AdaBoost', 'GradBoost','GaussianProcess', 'KNN', 'LR', 'NaiveBayes', 'RandomForest', 'SVM']
target_metric_name = evaluate.SENSITIVITY
target_metric_value = 0.8
ci_level = 0.95
metrics_output_file = os.path.join(data_dir, 'interim', 'scoring_metrics_fixed_{0}_{1}.csv'.
                                   format(target_metric_name,target_metric_value))

metrics_ranges_output_file = os.path.join(data_dir, 'interim', 'scoring_metrics_ranges_fixed_{0}_{1}.csv'.
                                   format(target_metric_name,target_metric_value))
# ************************* GLOBAL PARAMETERS END *************************************
def loaddata(file):
    with open(file,'r') as f:
        all_data = []
        for line in f.readlines():
            data = [float(x) for x in line.split(",")]
            all_data.append(data)
        return all_data

line = "model,acc,acc_std,acc_cil,acc_cih,f1,f1_std,f1_cil,f1_cih,sensitivity,sensitivity_std,sensitivity_cil,sensitivity_cih," \
       "specificity,specificity_std,specificity_cil,specificity_cih,precision,precision_std,precision_cil,precision_cih," \
       "npv,npv_std,npv_cil,npv_cih\n"
slog.log_line(line, metrics_output_file)

range_line = "model,acc,acc_low,acc_high,f1,f1_low,f1_high,sensitivity,sensitivity_low,sensitivity_high," \
       "specificity,specificity_low,specificity_high,precision,precision_low,precision_high," \
       "npv,npv_low,npv_high\n"
slog.log_line(range_line, metrics_ranges_output_file)

for fp in file_prefixes:
    probs = loaddata(_prob_file.format(fp))
    targs = loaddata(_targ_file.format(fp))
    acc, f1, sen, spec, precis, npv = evaluate.compute_metrics(targs, probs, target_metric_value, target_metric_name)
    scores = [acc, f1, sen, spec, precis, npv]
    line = "{0},".format(fp)
    range_line = "{0},".format(fp)
    for score in scores:
        m, s, cil, cih = mathx.mean_confidence_interval(score, ci_level)
        line="{0}{1},{2},{3},{4},".format(line, m,s,cil,cih)
        low = np.min(score)
        high = np.max(score)
        range_line="{0}{1},{2},{3},".format(range_line, m, low, high)
    line = "{0}\n".format(line[0:-1])
    range_line="{0}\n".format(range_line[0:-1])
    slog.log_line(line, metrics_output_file)
    slog.log_line(range_line, metrics_ranges_output_file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\RR\\Documents\\Python Scripts\\sepsis\\data\\results\\PATH/TO/PREDICTION/PROBABILITES/FILE\\prediction_probabilities\\AdaBoost_pred_probs.csv'